## 10.1 Saving Plots

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

fires = pd.read_csv("data/VIIRSNDE_global2018312.v1.0.txt")
bins10MW = np.arange(0, 500, 10)

In [2]:
plt.figure(figsize=[5,5]) 
plt.hist(fires['frp(MW)'],  bins=bins10MW)

plt.savefig('histogram.png',  bbox_inches='tight') 
plt.close()

## 10.2 Exporting Data

In [3]:
name = ['GOES-16', 'IceSat-2', 'Himawari']
agency = ['NOAA', 'NASA', 'JAXA']
orbit = ['GEO', 'LEO', 'GEO']

In [4]:
import csv

with open('satellite.txt','w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    
    # adds a header
    writer.writerow(['name', 'agency', 'orbit'])
    
    # Writes each element as a single row
    for i in range(0, len(name)):
        writer.writerow([name[i], agency[i], orbit[i]])

In [5]:
satellites = pd.DataFrame({'name': name,
                   'agency': agency,
                   'orbit': orbit})

satellites.to_csv('satellites.csv',index=False)

### 10.2.2 Pickling

In [6]:
import pickle as pkl
pkl.dump(satellites, open("satellites.p", "wb"))

In [7]:
satellites2 = pkl.load(open("satellites.p", "rb"))
satellites2

,name,agency,orbit
0,GOES-16,NOAA,GEO
1,IceSat-2,NASA,LEO
2,Himawari,JAXA,GEO


### 10.2.3 NumPy binary files

In [8]:
np.savez('satnames', name=name, agency=agency, orbit=orbit)

In [9]:
npzfile = np.load('satnames.npz')
npzfile.files

['name', 'agency', 'orbit']

In [10]:
npzfile['name']

array(['GOES-16', 'IceSat-2', 'Himawari'], dtype='<U8')

### 10.2.4 NetCDF

In [11]:
from netCDF4 import Dataset
from glob import glob

In [12]:
# Get file names, exit if no files are found
filenames = glob('data/aod/JRR-AOD_v2r0_j01_s2019*.nc')

# Import datasets
file_id = Dataset(filenames[0], mode='r')

In [13]:
for var in ["Latitude", "Longitude", "AOD550", "QCAll"]:
    print(var, file_id.variables[var].dimensions)

Latitude ('Rows', 'Columns')
Longitude ('Rows', 'Columns')
AOD550 ('Rows', 'Columns')
QCAll ('Rows', 'Columns')


In [14]:
for dim in ['Columns', 'Rows']:
    print(dim, len(file_id.dimensions[dim]))

Columns 3200
Rows 768


In [15]:
cols = len(file_id.dimensions['Columns'])
rows = len(file_id.dimensions['Rows'])
num_files = len(filenames)
file_id.close()

print(rows, cols, num_files)

768 3200 3


In [16]:
# Create output file name
output_filename = "JRR-AOD-combined.nc"

# Open a new netCDF file 
rootgrp = Dataset(output_filename, "w", format="NETCDF4")
rootgrp.description = "Combined NOAA-20 AOD swaths for Alberta Fire"

In [17]:
# Format the latitude and longitude variables
lat = rootgrp.createDimension("lat", rows*num_files)
lon = rootgrp.createDimension("lon", cols)

In [18]:
latitudes = rootgrp.createVariable("Latitude","f4", ("lat","lon"), \
           zlib = True, least_significant_digit = 2)
longitudes = rootgrp.createVariable("Longitude","f4", ("lat","lon"), \
           zlib = True, least_significant_digit = 2)
variable = rootgrp.createVariable('AOD550', "f4", ("lat","lon"), \
           zlib = True, least_significant_digit = 2, fill_value = -999.9)

In [19]:
latitudes.units = "degrees north"
longitudes.units = "degrees east"

In [20]:
var = np.zeros((rows*num_files, cols))
lats = np.zeros((rows*num_files, cols))
lons = np.zeros((rows*num_files, cols))

for filenum, filename in enumerate(filenames):
    file_id = Dataset(filename, mode='r')
    
    i1 = rows*filenum
    i2 = rows*(filenum+1)
    
    # Get coordinates
    lats[i1:i2, 0:cols] = file_id.variables['Latitude'][:,:]
    lons[i1:i2, 0:cols] = file_id.variables['Longitude'][:,:]
        
    # Create and fill variables
    value = file_id.variables['AOD550'][:,:]

    # Simple binary quality flag    
    dqf = file_id.variables[ 'QCAll'][:,:]
    value[ dqf != 0 ] = -999.9
    
    var[i1:i2, :] = value[:,:]

    file_id.close()

In [21]:
# Populate lat/lon/var
latitudes[:,:] = lats
longitudes[:,:] = lons
variable[:,:] = var

In [22]:
# Close original/modified netCDF file
rootgrp.close()

Using Xarray to create netCDF files 

In [23]:
import xarray as xr

fname = 'data/aod/JRR-AOD*.nc'
aod = xr.open_mfdataset(fname, concat_dim='Rows', combine='nested')
filtered = aod['AOD550'].where(aod['QCAll'] == 0)

In [24]:
filtered.to_netcdf('JRR-AOD-xarray-combined.nc')

In [25]:
del(filtered.attrs['units'])